<a href="https://colab.research.google.com/github/migueleaferreira/sd/blob/main/se%C3%A7%C3%A3o_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import socket
import threading
import json
import time

# --- Parte do Servidor Simulado em um Sistema Distribuído ---
# Esta classe representa um nó do sistema distribuído que oferece um recurso (lista de nomes)
# e um serviço para verificá-lo, similar a um servidor em uma arquitetura cliente-servidor [10, 11].
class ServidorSimulado:
    def __init__(self, host='127.0.0.1', port=12345):
        self.host = host
        self.port = port
        # Simulando um recurso compartilhado: uma lista de nomes [1]
        self.dados_recurso = [
            "Alice", "Bob", "Alice", "Charlie", "Bob", "Alice", "David", "Eve", "Alice", "Fernanda", "Pedro", "Ana", "Miguel"
        ]
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) # Cria um socket [12, 13]
        self.server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.server_socket.bind((self.host, self.port)) # Atrela o socket a um endereço e porta (primitiva bind) [12, 14]
        self.server_socket.listen(5) # Coloca o socket em modo de escuta para conexões (primitiva listen) [12, 15]
        self.running = True
        print(f"✔️ [Servidor] Servidor simulado iniciado em {self.host}:{self.port}...")

    def verificar_nomes(self, nome_procurado):
        """
        Método que simula a 'Chamada de Procedimento Remoto' (RPC) [9, 16].
        Ele conta a ocorrência de um nome nos dados do recurso compartilhado.
        """
        print(f"🔄 [Servidor] Recebida solicitação para verificar o nome: '{nome_procurado}'")
        count = self.dados_recurso.count(nome_procurado)
        return count

    def handle_client(self, client_socket, addr):
        """
        Lida com a comunicação individual com um cliente.
        Cada cliente é atendido em um thread separado para simular concorrência [17].
        """
        print(f"🤝 [Servidor] Conexão aceita de {addr}") # Conexão aceita (primitiva accept) [12, 15]
        try:
            # Loop para receber e responder requisições do cliente
            while self.running:
                client_socket.settimeout(30.0) # Timeout para a conexão do cliente
                data = client_socket.recv(1024).decode('utf-8') # Recebe dados (primitiva receive) [12, 18]
                if not data:
                    break

                request = json.loads(data) # Desserializa a requisição JSON
                method = request.get('method')
                args = request.get('args', [])

                if method == 'verificarNomes':
                    nome_procurado = args if args else ""
                    resultado = self.verificar_nomes(nome_procurado) # Executa o 'procedimento remoto'
                    response = {"result": resultado}
                    client_socket.sendall(json.dumps(response).encode('utf-8')) # Envia resposta (primitiva send) [12, 18]
                else:
                    response = {"error": "Método não encontrado no servidor."}
                    client_socket.sendall(json.dumps(response).encode('utf-8'))

        except socket.timeout:
            print(f"⚠️ [Servidor] Conexão com {addr} expirou.")
        except Exception as e:
            print(f"❌ [Servidor] Erro na comunicação com {addr}: {e}")
        finally:
            client_socket.close() # Fecha o socket do cliente (primitiva close) [12, 18]
            print(f"🚪 [Servidor] Conexão com {addr} encerrada.")

    def start(self):
        """Inicia o servidor e aguarda por conexões de clientes."""
        self.server_socket.settimeout(1.0) # Define um timeout para o accept para poder verificar 'self.running'
        while self.running:
            try:
                client_socket, addr = self.server_socket.accept() # Aguarda e aceita uma conexão [12, 15]
                client_handler = threading.Thread(target=self.handle_client, args=(client_socket, addr))
                client_handler.daemon = True # Torna o thread daemon para que ele encerre com o programa principal
                client_handler.start()
            except socket.timeout:
                continue # Continua aguardando se não houver conexão
            except Exception as e:
                if self.running:
                    print(f"❌ [Servidor] Erro ao aceitar conexão: {e}")
                break

    def stop(self):
        """Para o servidor simulado."""
        print("🛑 [Servidor] Encerrando o servidor simulado...")
        self.running = False
        self.server_socket.close() # Fecha o socket principal do servidor [12, 18]
        print("✅ [Servidor] Servidor simulado encerrado.")

# --- Parte do Cliente Simulado em um Sistema Distribuído ---
# Esta classe representa um nó que faz requisições a um serviço em outro nó.
class ClienteSimulado:
    def __init__(self, host='127.0.0.1', port=12345):
        self.host = host
        self.port = port
        print(f"🚀 [Cliente] Cliente simulado configurado para {self.host}:{self.port}")

    def chamar_verificar_nomes(self, nome):
        """
        Conecta ao servidor e 'chama' o método remoto 'verificarNomes' [7].
        """
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) # Cria um socket [12, 18]
        try:
            client_socket.connect((self.host, self.port)) # Tenta estabelecer conexão (primitiva connect) [12, 18]
            print(f"🔗 [Cliente] Conectado ao servidor em {self.host}:{self.port}")

            # Prepara a requisição RPC como JSON
            request = {"method": "verificarNomes", "args": [nome]}
            client_socket.sendall(json.dumps(request).encode('utf-8')) # Envia a requisição JSON [12, 18]
            print(f"📤 [Cliente] Enviou solicitação para verificar o nome: '{nome}'")

            # Recebe a resposta do servidor
            response_data = client_socket.recv(1024).decode('utf-8') # Recebe a resposta (primitiva receive) [12, 18]
            response = json.loads(response_data) # Desserializa a resposta JSON

            if "result" in response:
                print(f"✅ [Cliente] O nome '{nome}' aparece {response['result']} vezes no recurso compartilhado.")
            else:
                print(f"❌ [Cliente] Erro na resposta do servidor: {response.get('error', 'Desconhecido')}")

        except ConnectionRefusedError:
            print(f"⚠️ [Cliente] Não foi possível conectar ao servidor em {self.host}:{self.port}. Certifique-se de que o servidor está rodando.")
        except Exception as e:
            print(f"❌ [Cliente] Ocorreu um erro na comunicação: {e}")
        finally:
            client_socket.close() # Fecha o socket do cliente (primitiva close) [12, 18]
            print(f"🚪 [Cliente] Conexão do cliente encerrada.")

# --- Função Principal para Executar a Simulação no Google Colab ---
def run_distributed_simulation():
    server_host = '127.0.0.1' # Endereço de loopback para comunicação dentro da mesma VM [19]
    server_port = 12345 # Uma porta arbitrária e alta para evitar conflitos [19]

    print("=== Iniciando Simulação de Sistema Distribuído (Colab) ===")

    # 1. Iniciar o servidor em um thread separado
    # Isso permite que o servidor escute por conexões enquanto o resto do código executa [17].
    server = ServidorSimulado(host=server_host, port=server_port)
    server_thread = threading.Thread(target=server.start, daemon=True)
    server_thread.start()
    time.sleep(1.5) # Dá um pequeno tempo para o servidor inicializar completamente

    # 2. Clientes realizam requisições ao servidor
    print("\n--- Clientes fazendo requisições ---")

    # Primeiro cliente buscando "Alice"
    client1 = ClienteSimulado(host=server_host, port=server_port)
    client1.chamar_verificar_nomes("Alice")
    time.sleep(0.5)

    # Segundo cliente buscando "Bob"
    client2 = ClienteSimulado(host=server_host, port=server_port)
    client2.chamar_verificar_nomes("Bob")
    time.sleep(0.5)

    # Terceiro cliente buscando um nome que não existe no recurso
    client3 = ClienteSimulado(host=server_host, port=server_port)
    client3.chamar_verificar_nomes("Joana")
    time.sleep(0.5)

    # Quarto cliente buscando outro nome
    client4 = ClienteSimulado(host=server_host, port=server_port)
    client4.chamar_verificar_nomes("Miguel")
    time.sleep(0.5)

    # 3. Encerrar o servidor (opcional, pois é daemon e o ambiente Colab o encerraria)
    # server.stop()
    # server_thread.join() # Aguarda o thread do servidor terminar

    print("\n=== Simulação de sistema distribuído concluída (verificar output acima) ===")

# Garante que a função de simulação seja chamada quando o script é executado
if __name__ == '__main__':
    run_distributed_simulation()

=== Iniciando Simulação de Sistema Distribuído (Colab) ===
✔️ [Servidor] Servidor simulado iniciado em 127.0.0.1:12345...

--- Clientes fazendo requisições ---
🚀 [Cliente] Cliente simulado configurado para 127.0.0.1:12345
🔗 [Cliente] Conectado ao servidor em 127.0.0.1:12345
📤 [Cliente] Enviou solicitação para verificar o nome: 'Alice'
🤝 [Servidor] Conexão aceita de ('127.0.0.1', 52308)
🔄 [Servidor] Recebida solicitação para verificar o nome: '['Alice']'
✅ [Cliente] O nome 'Alice' aparece 0 vezes no recurso compartilhado.
🚪 [Cliente] Conexão do cliente encerrada.
🚪 [Servidor] Conexão com ('127.0.0.1', 52308) encerrada.
🚀 [Cliente] Cliente simulado configurado para 127.0.0.1:12345
🔗 [Cliente] Conectado ao servidor em 127.0.0.1:12345
📤 [Cliente] Enviou solicitação para verificar o nome: 'Bob'
🤝 [Servidor] Conexão aceita de ('127.0.0.1', 53054)
🔄 [Servidor] Recebida solicitação para verificar o nome: '['Bob']'
✅ [Cliente] O nome 'Bob' aparece 0 vezes no recurso compartilhado.
🚪 [Cliente] C